In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
input_data=np.load("input_data_emotion.npy")
label=np.load("label.np.npy")

In [3]:
input_data.shape

(35887, 48, 48, 3)

In [4]:
label=np.array(label)
input_data= (input_data*(1/256)-0.5)/0.5

In [5]:
input_data=torch.FloatTensor(input_data)
label=torch.LongTensor(label)

In [6]:
input_data=input_data.transpose(1,3)

In [7]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
dataset = TensorDataset(input_data,label)

In [8]:
trainset,valset=torch.utils.data.random_split(dataset,[30887,5000])
trainset,testset=torch.utils.data.random_split(trainset,[25887,5000])

partition={'train':trainset,'val':valset,'test':testset}

In [9]:
class AlexNet_test(nn.Module):
    
    def __init__(self):
        super(AlexNet_test,self).__init__()
        self.feature=nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=2),# 64 26 26 => kernel remake
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2),# 64 13 13 => kernel remake
        nn.Conv2d(64, 192, kernel_size=4, padding=2), # 192, 13, 13 => kernel remake
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=2,stride=2), # 192 7 7 => kernel remake
        nn.Conv2d(192, 384, kernel_size=3, padding=1), # 384 7 7 => kernel remake
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 256, kernel_size=3, padding=1), # 256 7 7
        nn.ReLU(inplace=True),
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2) # 256 6 6 
        )
        self.avgpool=nn.AdaptiveAvgPool2d((3,3)) # 256 6 6 (6,6)은 output size이다.
        # 평균을 구하는 것으로 반올림 정수가 나옴.
        self.classifier = nn.Sequential(
        nn.Dropout(),
        nn.Linear(256*3*3,1024),  # 9216=>4096, 256 36
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(1024,1024),
        nn.ReLU(inplace=True),
        nn.Linear(1024,7),
        
        )
    def forward(self, x):
        x=self.feature(x)
        x=self.avgpool(x)
        x=torch.flatten(x,1) # ex => 2,2,2 => 2,4
        x=self.classifier(x)
        return x

In [10]:
def dimension_check():
    net=AlexNet_test()
    x=torch.randn(1,3,48,48)
    y=net(x)
    print(y.size())

In [11]:
dimension_check()

torch.Size([1, 7])


In [12]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=64,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [13]:
def test(net, partition):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=64, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [14]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=64, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [15]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [16]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net=AlexNet_test()
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(100):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
test_acc = test(net,partition)
test_acc=round(test_acc,2)

  3%|▎         | 14/405 [00:00<00:02, 134.95it/s]

time : 3.50, train_loss : 1.81, train_acc : 24.85,val_acc : 24.48,val_loss : 1.80


  3%|▎         | 14/405 [00:00<00:02, 131.61it/s]

time : 3.16, train_loss : 1.80, train_acc : 25.06,val_acc : 24.48,val_loss : 1.80


  3%|▎         | 14/405 [00:00<00:02, 135.53it/s]

...

  3%|▎         | 14/405 [00:00<00:02, 132.97it/s]

time : 3.24, train_loss : 0.31, train_acc : 89.70,val_acc : 43.82,val_loss : 2.74


  0%|          | 0/79 [00:00<?, ?it/s]

time : 3.21, train_loss : 0.33, train_acc : 89.26,val_acc : 44.60,val_loss : 2.45


100%|██████████| 79/79 [00:00<00:00, 288.86it/s]


In [17]:
np.save("/home/lab/양창희/Emotion Project/test_list_file/Alex_train_loss.npy",train_loss)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Alex_val_loss.npy",val_loss)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Alex_train_acc.npy",train_acc)
np.save("/home/lab/양창희/Emotion Project/test_list_file/Alex_val_acc.npy",val_acc)